In [19]:
import LinearAlgebra
const la = LinearAlgebra


LinearAlgebra

# Projection Methods


Example

In [20]:
X = [1 2. 1;
     2 4. 1;
     3 -4 2;
     1 1. 1]

4×3 Matrix{Float64}:
 1.0   2.0  1.0
 2.0   4.0  1.0
 3.0  -4.0  2.0
 1.0   1.0  1.0

### Gram Schimdt

$X =[x_1, x_2, ..., x_r]\in \mathbb{R}^{n\times r}, n>=r$ matrix with independent column vectors. 

In [21]:
function gram_schmidt(X::Matrix{Float64})
    # input: X is a nxr matrix with columns linear independent
    n, r = size(X)
    R = zeros(Float64, (r, r))
    Q = zeros(Float64, (n, r))

    R[1, 1] = la.norm(X[:, 1])

    if R[1, 1] < eps()
        return nothing
    end
    Q[:, 1] = X[:, 1]/R[1, 1]
    
    for j = 2:r
        R[1:j-1, j] = transpose(Q[:, 1:j-1])*X[:, j]    # [<q1, xj>, ..., <q[j-1], xj>]T
        q = X[:, j] - Q[:, 1:j-1]*R[1:j-1, j]           # [q1*r[1,j], ..., q[j-1]*r[j-1,j]]T
        R[j, j] = la.norm(q)
        if R[j, j] < eps()
            break
        end
        Q[:, j] = q/R[j, j]
    end
    return Q, R
end


gram_schmidt (generic function with 1 method)

In [22]:
Q, R = gram_schmidt(X)
display(Q)
display(R)

4×3 Matrix{Float64}:
 0.258199   0.340064   0.612427
 0.516398   0.680129  -0.515067
 0.774597  -0.62528   -0.0596724
 0.258199   0.175517   0.596724

3×3 Matrix{Float64}:
 3.87298  -0.258199   2.58199
 0.0       6.07728   -0.0548491
 0.0       0.0        0.574739

In [23]:
function modified_gram_schmidt(X::Matrix{Float64})
    # input: X is a nxr matrix with columns linear independent
    n, r = size(X)
    R = zeros(Float64, (r, r))
    Q = zeros(Float64, (n, r))

    R[1, 1] = la.norm(X[:, 1])
    
    if R[1, 1] < eps()
        return nothing
    end
    Q[:, 1] = X[:, 1]/R[1, 1]

    for j = 2:r
        q = X[:, j]
        for i = 1:j-1
            R[i, j] = la.dot(q, Q[:, i])
            q = q - R[i, j]*Q[:, i]
        end
        R[j, j] = la.norm(q)
        if R[j, j] < eps()
            break
        end
        Q[:, j] = q/R[j, j]
    end
    return Q, R
end


modified_gram_schmidt (generic function with 1 method)

In [24]:
Q, R = modified_gram_schmidt(X)
display(Q)
display(R)
Q * R

4×3 Matrix{Float64}:
 0.258199   0.340064   0.612427
 0.516398   0.680129  -0.515067
 0.774597  -0.62528   -0.0596724
 0.258199   0.175517   0.596724

3×3 Matrix{Float64}:
 3.87298  -0.258199   2.58199
 0.0       6.07728   -0.0548491
 0.0       0.0        0.574739

4×3 Matrix{Float64}:
 1.0   2.0  1.0
 2.0   4.0  1.0
 3.0  -4.0  2.0
 1.0   1.0  1.0

### Householder

$X=[x_1, x_2, ..., x_m]\in \mathbb{R}^{n\times m}, Q \in \mathbb{R}^{n\times n} \wedge R \in \mathbb{R}^{m\times m}$ a upper triangular matrix. This method return $X=QR$ decomposition.

In [25]:
function householder(X:: Matrix{Float64})
    n, m = size(X)
    I = Matrix{Float64}(la.I, n, n)
    R = zeros(n, m) # [R 0]'
    R[:, 1] = X[:, 1]
    Q = I
    for k = 1:m
        if k > 1
            R[:, k] = Q * X[:, k]
        end
        beta = sign(R[k, k]) * la.norm(R[k:end, k])
        z = zeros(n)
        z[k] = beta + R[k, k]
        for i = k+1:n
            z[i] = R[i, k]
        end
        w = z / la.norm(z)
        P = I - 2 * w * w'
        # P*R[:,k] makes 0 the k + 1 next elements
        R[:, k] = P * R[:, k]
        Q = P * Q
    end
    return Q', R
end

householder (generic function with 1 method)

In [26]:
Q, R = householder(X)
display(Q)
display(R) 
Q * R

4×4 adjoint(::Matrix{Float64}) with eltype Float64:
 -0.258199  -0.340064   0.612427   -0.665299
 -0.516398  -0.680129  -0.515067    0.0739221
 -0.774597   0.62528   -0.0596724  -0.0739221
 -0.258199  -0.175517   0.596724    0.739221

4×3 Matrix{Float64}:
 -3.87298       0.258199     -2.58199
  4.16334e-17  -6.07728       0.0548491
  4.16334e-16  -5.48173e-16   0.574739
 -1.11022e-16   0.0           0.0

4×3 Matrix{Float64}:
 1.0   2.0  1.0
 2.0   4.0  1.0
 3.0  -4.0  2.0
 1.0   1.0  1.0